In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install libraries
!pip install rioxarray sahi ultralytics geopandas shapely pillow

In [ ]:
# Impoort libraries
import numpy as np
import rioxarray
import geopandas as gpd
from shapely.geometry import shape
from shapely.ops import unary_union
from rasterio.features import shapes
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction
from PIL import Image
import tempfile, os

In [ ]:
# Read GeoTIFF + Reproject to UTM
def read_geotiff(tif_path):
    xds = rioxarray.open_rasterio(tif_path)

    # Band count validation
    if xds.shape[0] < 3:
        raise ValueError(f"GeoTIFF only has {xds.shape[0]} bands, minimum 3 required")

    # Auto-detect UTM zone
    center_lon = float((xds.x.min() + xds.x.max()) / 2)
    center_lat = float((xds.y.min() + xds.y.max()) / 2)
    zone = int((center_lon + 180) / 6) + 1
    epsg = 32600 + zone if center_lat >= 0 else 32700 + zone
    print(f"Auto UTM: EPSG:{epsg}")

    # Reproject to UTM
    xds_utm = xds.rio.reproject(f"EPSG:{epsg}")

    meta = {
        'transform': xds_utm.rio.transform(),
        'crs': xds_utm.rio.crs,
        'height': xds_utm.rio.height,
        'width': xds_utm.rio.width,
    }

    # Convert to uint8
    image = np.transpose(xds_utm.values[:3], (1, 2, 0))
    if image.dtype != np.uint8:
        image = ((image - image.min()) /
                 (image.max() - image.min() + 1e-8) * 255).astype(np.uint8)

    # Save tmp jpg
    with tempfile.NamedTemporaryFile(suffix=".jpg", delete=False) as f:
        tmp_path = f.name
    Image.fromarray(image).save(tmp_path, quality=95)

    return tmp_path, meta

In [ ]:
# 2.Inference with SAHI
def run_sahi(img_path, model_path, conf=0.25):
    model = AutoDetectionModel.from_pretrained(
        model_type="ultralytics",
        model_path=model_path,
        confidence_threshold=conf,
        device="cuda",  # replace "cpu" if there is no GPU
    )

    result = get_sliced_prediction(
        img_path, model,
        slice_height=640, slice_width=640,
        overlap_height_ratio=0.2, overlap_width_ratio=0.2,
        perform_standard_pred=False,
        postprocess_type="NMM",
        postprocess_match_threshold=0.5,
        verbose=1,
    )

    return result

In [ ]:
# Convert results to GeoJSON
def predictions_to_geojson(result, meta, output_path="output/predictions.geojson"):
    records = []

    for pred in result.object_prediction_list:
        if pred.mask is None:
            continue

        bool_mask = pred.mask.bool_mask.astype(np.uint8)

        for geom, val in shapes(bool_mask, mask=bool_mask, transform=meta['transform']):
            if val == 0:
                continue
            records.append({
                'geometry': shape(geom),
                'class': pred.category.name,
                'confidence': round(pred.score.value, 4)
            })

    if not records:
        print("No objects detected")
        return gpd.GeoDataFrame()

    gdf = gpd.GeoDataFrame(records, crs=meta['crs'])
    gdf['area_m2'] = gdf.geometry.area.round(2)

    output_dir = os.path.dirname(output_path)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    gdf.to_file(output_path, driver="GeoJSON")
    print(f"Save: {output_path} | Total objects: {len(gdf)}")

    return gdf

In [ ]:
# Running Infrance
tmp_path, meta = read_geotiff("input.tif")
try:
    result = run_sahi(tmp_path, "best.pt", conf=0.25)
    gdf = predictions_to_geojson(result, meta, "output/predictions.geojson")
finally:
    os.remove(tmp_path) 

# Summery
if not gdf.empty:
    print(gdf.groupby('class')['area_m2'].agg(['count', 'sum', 'mean']).round(2))

In [ ]:
# Post Processing
def postprocess_geojson(input_path, output_path,
                         min_area_m2=150,      # delete polygon 150 <  m²
                         buffer_dist=1.0):      # merge distance in meters

    gdf = gpd.read_file(input_path)

    # Remove very small fragments
    gdf = gdf[gdf['area_m2'] >= min_area_m2].copy()
    print(f"Setelah filter kecil: {len(gdf)} objek")

    # Small buffer → adjacent merge → reverse buffer
    gdf['geometry'] = gdf.geometry.buffer(buffer_dist)   # expand
    gdf['geometry'] = gdf.geometry.buffer(-buffer_dist)  # shrink balik

    # Dissolve overlapping/touching polygons
    gdf_dissolved = gdf.dissolve(by='class').reset_index()

    # Break it back into individual polygons (explode multipolygon)
    gdf_dissolved = gdf_dissolved.explode(index_parts=False).reset_index(drop=True)

    # Recalculate the area after dissolving
    gdf_dissolved['area_m2'] = gdf_dissolved.geometry.area.round(2)

    # Filter again after dissolving
    gdf_dissolved = gdf_dissolved[gdf_dissolved['area_m2'] >= min_area_m2]

    gdf_dissolved.to_file(output_path, driver="GeoJSON")
    print(f"✅ Done: {len(gdf_dissolved)} objec → {output_path}")

    return gdf_dissolved

# Running
gdf_clean = postprocess_geojson(
    input_path="/content/output/predictions.geojson",
    output_path="/content/drive/MyDrive/yolo/crop_map.v5i.yolov11/infrance/predictions_clean.geojson",
    min_area_m2=150,    # adjust — minimum valid rice field area
    buffer_dist=1.0     # adjust — the bigger the more aggressive the merge
)

print(gdf_clean[['class', 'area_m2']].describe().round(2))